In [ ]:
#Image classifer with logistic regression
#images shape(width,height,3(rgb))

import numpy as np
import scipy 
import os
import matplotlib.pyplot as plot
import math
from scipy import ndimage

# TO-DO
# fetch a ton more images, of your own choice / category 
# create a train and a test set <---- write code for this 

In [ ]:
# Global Constants
side_length = 650
image_path = "./untitled folder/"
test_ratio = .2

In [ ]:
#
test0 = "./untitled folder/" + "cow0.jpg" #standardize file extensions
test1 = "./untitled folder/" + "notCow0.jpg" 
test0Array = scipy.ndimage.imread(test0,flatten=False)
test1Array = scipy.ndimage.imread(test1,flatten=False) 
print("Shape of Test Image 0:", np.shape(test0Array))
print("Shape of Test Image 1:", np.shape(test1Array))

# Resize input image arrays 
test0Array_resized = scipy.misc.imresize(test0Array,size=(side_length,side_length,3))
test1Array_resized = scipy.misc.imresize(test1Array,size=(side_length,side_length,3))

In [ ]:
# Read in input image files into arrays, validate inputs
images = os.listdir(image_path)

# First, build dataset for training data
examples = []
labels = []
for image in images:
    if image.split(".")[-1] != "DS_Store":
        cur_label = 0
        if image[0] == "c":
            cur_label = 1
        labels += [cur_label]
        cur_image_path = image_path + image
        cur_array = scipy.ndimage.imread(cur_image_path,flatten=False) #reads image as numpy array (lenght,height,3)
        cur_array_resized = scipy.misc.imresize(cur_array,size=(side_length,side_length,3)) #resizes images to a uniform shape
        examples += [cur_array_resized] 
# print(labels)

print("Shape of train_examples:",np.shape(examples)) #should be (len(images),side_length,side_length,3)

# Then, build dataset for train and test data...
number_test_examples = int(len(examples)*test_ratio)
test_examples = examples[:number_test_examples]
train_examples = examples[number_test_examples:]

train_examples = np.array(train_examples) #converts the list of examples to an array
test_examples = np.array(test_examples) 


# Next, we flatten our train and test examples
# .T => transposes our inputs
# (a, b, c, d) ----> (b * c * d, a)

flattened_train_examples = train_examples.reshape(train_examples.shape[0], -1).T
flattened_test_examples = test_examples.reshape(test_examples.shape[0], -1).T
# print(type(train_examples[0]))
# flattened_train_examples = train_examples.reshape(train_examples[0],-1).T
#fattened_train_examples = train_examples.flatten()
print(np.shape(flattened_train_examples))
print(np.shape(flattened_test_examples))

# print("Shape of test0Array_resized:",np.shape(test0Array_resized))

# Standardize color values of the image 
standardized_train_examples = flattened_train_examples/255 #225 is the maximum rgb value/ This is done to decrease varaince in inputs thus more efficint
standardized_test_examples = flattened_test_examples/255
print(standardized_train_examples)


# Create labels for data
# labels = [1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0]
labels_train  = np.array(labels[:21])
labels_test = np.array(labels[21:])
# Randomize and shuffle labels/data

In [ ]:
def sigmoid(x):
    output = 1.0/(1.0+np.exp(-x))
    return output
    
sigmoid(10)
# write test cases for this function so that it works for np arrays 

In [ ]:
def initialize_zeros(dimension): #dimension = number of examples
    zeroed_weights = np.zeros((dimension, 1))
    zeroed_baisies = 0. #baisies always start at 0 b/c they are developed as the model trains
    
    # asserts here, use isinstance()
    # check shape of zeroed weight
    assert(isinstance(zeroed_baisies, float) or isinstance(zeroed_baisies, int))
    
    return zeroed_weights, zeroed_baisies
    
initialize_zeros(10)
# write test cases for this function <--- use asserts, check that zeored_weights and zeored_baisies are of the right
# type. 

In [47]:
def propagate(X,Y,w,b):
    # b = scalar
    # w = (____, 1)
    # X = (____, examples)
    # Y = (1, examples)
    
    w_transposed = w.T
    num_of_examples = np.shape(X)[1]
#     print(np.shape(X))
#     print(np.shape(w_transposed))
    # format printing of shapes so that you can see the dot product
    dot_product = np.dot(w_transposed,X) #dot product of X and w ORDER MATTERS
#     print(np.shape(dot_product))
    z = dot_product + b
    a = sigmoid(z) #in begining should be 0
    loss = (-1/num_of_examples)*np.sum((1-Y)*np.log(1-a) + (Y *np.log(a))) #log likelyhood cost for logistic regression
    #
    dw = (1/num_of_examples) * np.dot(X,(a-Y).T)
    #
    db = (1/num_of_examples) * np.sum(a-Y) #np.sum collapes input
    #
    loss = np.squeeze(loss)
    #
    grads = {"dw":dw,"db":db}
    return grads,loss    


In [ ]:
w,b = initialize_zeros(np.shape(standardized_train_examples)[0])
propagate(standardized_train_examples,labels_train,w,b)


In [ ]:
def optimizer(w, b, X, Y, learning_rate, num_iterations, print_results=True):
    # b = scalar
    # w = (____, 1)
    # X = (____, examples)
    # Y = (1, examples)
    costs = []
    for i in range(num_iterations):
        grads, cost = propagate(X,Y,w,b)
        print(cost)
        dw = grads["dw"]
        db = grads["db"]
        w = w - (learning_rate * dw)
        b = b - (learning_rate * db)
        if i % 50 == 0:
            costs += [cost]
            print("cost #",i,":", costs)
    print("Finsihed :)" , costs )
    grads = {"dw":dw,"db":db}
    params = {"w":w,"b":b}
    return grads,params,costs

In [62]:
print(np.shape(labels_train),np.shape(standardized_train_examples))
print(w.shape,standardized_train_examples.shape,labels_train.shape)
# optimizer(w,b,standardized_train_examples,labels_train,.1,1000)

(21,) (1267500, 21)
(1267500, 1) (1267500, 21) (21,)


In [32]:
def predictor(X,w,b):
    # b = scalar
    # w = (____, 1)
    # X = (____, examples)
    w_transposed = w.T
    print(np.shape(w_transposed),np.shape(X))
#     new_w = w_transposed.reshape(X.shape[0], 1)
#     print(new_w.shape)
#     print(np.reshape(w_transposed, X[0]))
    prediction = (np.dot(w_transposed,X)) + b
    prediction = sigmoid(prediction)
    prediction *= 2.0
    prediction = np.floor(prediction)
    print(np.shape(prediction))
    return prediction
    


In [33]:
predict_examples = (standardized_train_examples)
# for item in predict_examples:
#     print(np.shape(item))
predictor(predict_examples,w,b)

(1, 1267500) (1267500, 21)
(1, 21)


In [58]:
def logisticRegression(train_X,train_Y,test_X,test_Y,learning_rate,epoch):
    # train_x = (image_width * image_width * 3, examples)
    # train_y = (1, examples)
    # test_x = (____, examples)
    # test_y = (1, examples)

    print(train_X.shape)
    num_of_examples =  np.shape(train_Y)[0]
    train_Y = train_Y.reshape(num_of_examples,1)
    train_Y = train_Y.T
    print(train_Y.shape)
#     assert()
    w,b = initialize_zeros(num_of_examples)
    print(w.shape)
    _, params,_ = optimizer(w, b, train_X, train_Y, learning_rate, epoch)
    w,b = params["w"], params["b"]
    prediction_train = predictor(train_X,w,b)
    accuracy_train = np.mean(np.abs(prediction_train - train_Y))
    prediction_test = predictor(test_X,w,b)
    accuracy_test = np.mean(np.abs(prediction_test - test_Y))
    print("Train Accuracy:", accuracy_train)
    print("Actual Accuracy:", accuracy_test)
    
    
    
    
    
    
    
    
    

In [59]:
logisticRegression(standardized_train_examples,labels_train,standardized_test_examples,labels_test,.1,1000)

(1267500, 21)
(1, 21)
(21, 1)


ValueError: shapes (1,21) and (1267500,21) not aligned: 21 (dim 1) != 1267500 (dim 0)